In [1]:
import numpy as np
import time
import pandas as pd

t_x = pd.read_csv("pa3_train_X.csv")
t_y = pd.read_csv("pa3_train_y.csv")
d_x = pd.read_csv("pa3_dev_X.csv")
d_y = pd.read_csv("pa3_dev_y.csv")

t_x.head()

,Gender,Age,Driving_License,Previously_Insured,Vehicle_Damage,Annual_Premium,Vintage,Region_Code_0,Region_Code_1,Region_Code_2,...,Policy_Sales_Channel_153,Policy_Sales_Channel_154,Policy_Sales_Channel_155,Policy_Sales_Channel_156,Policy_Sales_Channel_157,Policy_Sales_Channel_158,Policy_Sales_Channel_159,Policy_Sales_Channel_160,Policy_Sales_Channel_163,Intercept
0,0,0.296875,1,0,1,0.080294,0.968858,0,0,0,...,0,0,0,0,0,0,0,0,0,1.0
1,0,0.375000,1,0,1,0.093138,0.159170,0,0,0,...,0,0,0,0,0,0,0,0,0,1.0
2,0,0.625000,1,0,1,0.092671,0.757785,0,0,0,...,0,0,0,0,0,0,0,0,0,1.0
3,1,0.031250,1,0,1,0.044411,0.629758,0,0,0,...,0,0,0,0,0,0,0,0,0,1.0
4,1,0.484375,1,0,1,0.058486,0.183391,0,0,0,...,0,0,0,0,0,0,0,0,0,1.0


In [2]:
temp = t_x['Intercept']
t_x = t_x.drop('Intercept', 1)
t_x.insert(0, 'Intercept', temp)

temp = d_x['Intercept']
d_x = d_x.drop('Intercept', 1)
d_x.insert(0, 'Intercept', temp)

In [4]:
def gram_matrix(X_val, X2_val, p):
	X = np.asarray(X_val)
	X2 = np.asarray(X2_val)

	K = np.power((1 + np.dot(X, X2.T)), p)
	return K

def calculate_acc(X_val, y_train, y_val, Kernal, alpha):
	N = X_val.shape[0]

	X = np.asarray(X_val)
	y = np.asarray(y_val)
	y = y.flatten()

	y_tr = np.asarray(y_train)
	y_tr = y_tr.flatten()
	
	predict = (y == np.sign(np.dot(np.multiply(alpha, y_tr), Kernal)))
	acc = (float) (np.count_nonzero(predict)) / len(y)
	
	return acc * 100

#np.set_printoptions(threshold=np.inf)

In [5]:
def learning(t_x, t_y, d_x, d_y, max_itr, p):
	N = t_x.shape[0]
	Kernal = gram_matrix(t_x, t_x, p)
	VKernal = gram_matrix(t_x, d_x, p)

	alpha = np.zeros(N)
	
	u = np.zeros(N)

	X = np.asarray(t_x)
	y = np.asarray(t_y)
	y = y.flatten()

	X_v = np.asarray(d_x)
	y_v = np.asarray(d_y)
	y_v = y_v.flatten()

	itr_cnt = 0
	t = time.time()

	train_acc_vals = []
	valid_acc_vals = []

	
	while (itr_cnt < max_itr):
		itr_cnt += 1
		
		val = np.multiply(alpha.T, Kernal)
		u = np.dot(val, y)
		upd = np.multiply(u, y)

		alpha = alpha + 1 * np.where(upd <= 0, 1, 0)
		
		#for i in range(N):
		#	if upd[i] <= 0:
		#		alpha[i] = alpha[i] + 1
		
		train_acc = calculate_acc(t_x, t_y, t_y, Kernal, alpha)
		train_acc_vals.append(train_acc)
		
		valid_acc = calculate_acc(d_x, t_y, d_y, VKernal, alpha)
		valid_acc_vals.append(valid_acc)
		
		if(itr_cnt % 10 == 0):
			print("Train accuracy with p value "+ str(p) + " is : " + str(train_acc) + " itr_cnt : " + str(itr_cnt))
			print("Validation accuracy with p value " + str(p) + " is : " + str(valid_acc) + " itr_cnt : " + str(itr_cnt))
	
	elapsed = time.time() - t
	
	return (alpha, train_acc_vals, valid_acc_vals, elapsed)


In [ ]:
max_itr = 100

alpha, train_acc_vals, valid_acc_vals, elapsed = learning(t_x, t_y, d_x, d_y, max_itr, 1)

for itr in range(1, max_itr+1):
	alpha, train_acc_vals, valid_acc_vals, time_val = learning(t_x, t_y, d_x, d_y, itr, 1)
	if(itr % 10 == 0):
		print()
		print("time took : " + str(time_val) + " with p value : " + str(1) + " with iteration : " + str(itr))

Train accuracy with p value 1 is : 49.6318407960199 itr_cnt : 10
Validation accuracy with p value 1 is : 49.858585858585855 itr_cnt : 10
Train accuracy with p value 1 is : 75.75124378109453 itr_cnt : 20
Validation accuracy with p value 1 is : 75.87878787878788 itr_cnt : 20
Train accuracy with p value 1 is : 77.70149253731343 itr_cnt : 30
Validation accuracy with p value 1 is : 78.18181818181819 itr_cnt : 30
Train accuracy with p value 1 is : 75.87064676616916 itr_cnt : 40
Validation accuracy with p value 1 is : 76.1010101010101 itr_cnt : 40
Train accuracy with p value 1 is : 74.82587064676616 itr_cnt : 50
Validation accuracy with p value 1 is : 74.84848484848486 itr_cnt : 50
Train accuracy with p value 1 is : 49.62189054726368 itr_cnt : 60
Validation accuracy with p value 1 is : 49.898989898989896 itr_cnt : 60
Train accuracy with p value 1 is : 49.64179104477612 itr_cnt : 70
Validation accuracy with p value 1 is : 49.878787878787875 itr_cnt : 70
Train accuracy with p value 1 is : 49.64